In [17]:
import datetime 
import pandas as pd
import pymongo
import warnings
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from matplotlib.patches import FancyBboxPatch
import os
from datetime import datetime, timedelta
os.environ['CURL_CA_BUNDLE'] = ''
warnings.filterwarnings("ignore")
client = pymongo.MongoClient('10.128.96.25', 27017)
db = client["sentiment_news"]
sentiment_newsCollection = db["sentiment_newsCollection"]
sentiment_news_df = pd.DataFrame(list(sentiment_newsCollection.find()))

today = datetime.today().strftime('%Y-%m-%d')

one_week_dates = []
for day_index in range(0,7):
    one_week_dates.append((datetime.today() - timedelta(day_index)).strftime('%Y-%m-%d'))
    
months = []
for day_index_2 in range(7,30):
    months.append((datetime.today() - timedelta(day_index_2)).strftime('%Y-%m-%d'))

week_tickers_df = pd.DataFrame(columns = ['ticker', 'positive', 'negative', 'neutral'])

one_week_sentiments = sentiment_news_df[sentiment_news_df["date"].str.contains('|'.join(one_week_dates))== True]
one_week_sentiments = one_week_sentiments[one_week_sentiments['tickers'].astype(str) != '[]']
one_week_sentiments = one_week_sentiments.reset_index(drop=True)


for i in range(0,len(one_week_sentiments)):
    for ii in range(0,len(one_week_sentiments['tickers'][i])):
        tickers = (one_week_sentiments['tickers'][i][ii])
        positive = (one_week_sentiments['positive'][i][ii])
        negative = (one_week_sentiments['negative'][i][ii])
        neutral = (one_week_sentiments['neutral'][i][ii])
        new_row = {"ticker": tickers, "positive": positive, "negative": negative, "neutral": neutral}
        week_tickers_df = week_tickers_df.append(new_row, ignore_index=True)
        
        

week_tickers_df_avarage = week_tickers_df.groupby(['ticker']).mean()

month_tickers_df = pd.DataFrame(columns = ['ticker', 'positive', 'negative', 'neutral'])

month_sentiments = sentiment_news_df[sentiment_news_df["date"].str.contains('|'.join(months))== True]
month_sentiments = month_sentiments[month_sentiments['tickers'].astype(str) != '[]']
month_sentiments = month_sentiments.reset_index(drop=True)

for i in range(0,len(month_sentiments)):
    for ii in range(0,len(month_sentiments['tickers'][i])):
        tickers = (month_sentiments['tickers'][i][ii])
        positive = (month_sentiments['positive'][i][ii])
        negative = (month_sentiments['negative'][i][ii])
        neutral = (month_sentiments['neutral'][i][ii])
        new_row = {"ticker": tickers, "positive": positive, "negative": negative, "neutral": neutral}
        month_tickers_df = month_tickers_df.append(new_row, ignore_index=True)
        
        
month_tickers_df_avarage = month_tickers_df.groupby(['ticker']).mean()


week_tickers_df_avarage['status'] = np.nan
week_tickers_df_avarage['status'] = week_tickers_df_avarage['status'].astype(object)
# week_tickers_df_avarage = week_tickers_df_avarage.reset_index(drop=True)

for row_index in range(0,len(week_tickers_df_avarage)):
    sentiment = {'positive' : week_tickers_df_avarage['positive'][row_index], 'negative' : week_tickers_df_avarage['negative'][row_index], 'neutral' : week_tickers_df_avarage['neutral'][row_index]}
    # print((sentiment))
    inverse = [(value, key) for key, value in sentiment.items()]
    week_tickers_df_avarage['status'][row_index] = (max(inverse)[1])
    # compare = [(value, key) for key, value in sentiment.items()]
    # week_tickers_df_avarage['status'] = (max(sentiment))
    
month_tickers_df_avarage['status'] = np.nan
month_tickers_df_avarage['status'] = month_tickers_df_avarage['status'].astype(object)
# week_tickers_df_avarage = week_tickers_df_avarage.reset_index(drop=True)
# month_tickers_df_avarage


for row_index in range(0,len(month_tickers_df_avarage)):
    sentiment = {'positive' : month_tickers_df_avarage['positive'][row_index], 'negative' : month_tickers_df_avarage['negative'][row_index], 'neutral' : month_tickers_df_avarage['neutral'][row_index]}
    # print((sentiment))
    inverse = [(value, key) for key, value in sentiment.items()]
    month_tickers_df_avarage['status'][row_index] = (max(inverse)[1])
    # compare = [(value, key) for key, value in sentiment.items()]
    # week_tickers_df_avarage['status'] = (max(sentiment))
    
#########################################################################
selected_ticker = str(input('Ticker ismi : '))
selected_ticker_tt = selected_ticker
selected_ticker = ticker_dict[selected_ticker]
#########################################################################
# print(selected_ticker)


selected_month_row = month_tickers_df_avarage.loc[selected_ticker]
selected_week_row = week_tickers_df_avarage.loc[selected_ticker]

selected_ticker = selected_ticker_tt ####################################################3
if selected_week_row['status'] == 'negative':
    print(f"{selected_ticker} hakkındaki haberler son zamanlarda olumsuz ilerlemekte.")
elif selected_week_row['status'] == 'positive':
    print(f"{selected_ticker} hakkındaki haberler son zamanlarda olumlu ilerlemekte.")
else:
    print(f"{selected_ticker} hakkındaki haberler son zamanlarda ne olumlu ne olumsuz ilerlemekte.")
    
    
if selected_month_row['status'] == 'negative':
    print(f"{selected_ticker} hakkındaki haberler genel olarak olumsuz ilerlemekte.")
elif selected_month_row['status'] == 'positive':
    print(f"{selected_ticker} hakkındaki haberler genel olarak olumlu ilerlemekte.")
else:
    print(f"{selected_ticker} hakkındaki haberler genel olarak ne olumlu ne olumsuz ilerlemekte.")


MMM hakkındaki haberler son zamanlarda ne olumlu ne olumsuz ilerlemekte.
MMM hakkındaki haberler genel olarak olumsuz ilerlemekte.


In [4]:
print(month_tickers_df)

               ticker   positive  negative    neutral
0               BANKS  89.071408  0.967256   9.961338
1     GLOBAL PAYMENTS   4.047646  1.488369  94.463986
2                GOLD   3.304115  2.309310  94.386578
3         REAL ESTATE   3.304115  2.309310  94.386578
4          FINANCIALS  93.412751  2.820887   3.766356
...               ...        ...       ...        ...
1442            INTEL  63.616323  1.169666  35.214013
1443          INVESCO  63.616323  1.169666  35.214013
1444        MICROSOFT  90.437472  1.176141   8.386394
1445           NASDAQ  63.616323  1.169666  35.214013
1446           NVIDIA  90.437472  1.176141   8.386394

[1447 rows x 4 columns]


In [10]:
list(ticker_dict.keys())[list(ticker_dict.values()).index('MMM')]

ValueError: 'MMM' is not in list

In [12]:
ticker_dict['MMM']

'3M'

In [6]:
import datetime 
import pandas as pd
import pymongo
import warnings
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from matplotlib.patches import FancyBboxPatch
import os
from datetime import datetime, timedelta


ticker_names = pd.read_csv('ticker_names.csv')
ticker_dict = dict(zip(ticker_names.ticker, ticker_names.name))

ticker_values = list(ticker_dict.values())
ticker_keys = list(ticker_dict.keys())

In [7]:
ticker_keys

['A',
 'AA',
 'AAL',
 'AAPL',
 'ABBV',
 'ABC',
 'ABMD',
 'ABT',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'ADT',
 'AEE',
 'AEP',
 'AFL',
 'AIG',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALL',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ATVI',
 'AVGO',
 'AVY',
 'AXP',
 'AXTA',
 'AZEK',
 'AZO',
 'BA',
 'BAC',
 'BAM',
 'BAX',
 'BBWI',
 'BDX',
 'BEP',
 'BG',
 'BIIB',
 'BIO',
 'BJ',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMRN',
 'BMY',
 'BRK-B',
 'BSX',
 'BWA',
 'C',
 'CACI',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHTR',
 'CI',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'COUP',
 'CPB',
 'CPRT',
 'CRL',
 'CRM',
 'CSCO',
 'CSX',
 'CTAS',
 'CTLT',
 'CTRA',
 'CTSH',
 'CVS',
 'CVX',
 'D',
 'DAL',
 'DD',
 'DE',
 'DELL',
 'DFS',
 'DG',
 'DGX',
 'DHI',
 'DHR',
 '